In [9]:
import cv2
import numpy as np


In [10]:
faceClassif = cv2.CascadeClassifier("haarcascade_eye.xml")  # Cargamos el modelo de clasificacion de rostros 
image = cv2.imread("ojo.jpg")  #Leemos la imagen que vamos a reconocer 

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convertimos la imagen a blanco y negro 



In [11]:

faces = faceClassif.detectMultiScale(gray,
    scaleFactor=1.09,
    minNeighbors=5,
    minSize=(30,30),
    maxSize=(200,200))

for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cortado = [x,y,w,h]
    
print(cortado)
puntoDerecha= cortado[0] + cortado[2]
puntoAbajo = cortado[1] + cortado[3]
x1 = cortado[0] 
y1= cortado[1]

img = image[y1:puntoAbajo, x1:puntoDerecha]

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[51, 34, 138, 138]


In [12]:
faceClassif = cv2.CascadeClassifier("haarcascade_eye.xml")  # Cargamos el modelo de clasificacion de rostros 
# image = cv2.imread("ojo.jpg")  #Leemos la imagen que vamos a reconocer 

imgblur = cv2.medianBlur(img,5)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convertimos la imagen a blanco y negro 

# detect circles in the image
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20,
                            param1=50, param2=30, minRadius=0, maxRadius=0)

if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    

    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # dibujar circulo 
        cv2.circle(img, (i[0], i[1]), i[2], (0,255,0), 2)
        # dibujar centro
        cv2.circle(img, (i[0], i[1]), 2, (0,0,255), 3)

    cv2.imshow('detected circles', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
# Para mostrar la Imagen
cv2.imshow('image', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# Codigo para video en Vivo
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

faceClassif = cv2.CascadeClassifier('haarcascade_eye.xml')

while True:
	ret,frame = cap.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	faces = faceClassif.detectMultiScale(gray, 1.3, 5)

	for (x,y,w,h) in faces:
		cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)

	cv2.imshow('frame',frame)
	
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
cap.release()
cv2.destroyAllWindows()

error: ..\..\..\modules\imgproc\src\color.cpp:7456: error: (-215) scn == 3 || scn == 4 in function cv::ipp_cvtColor


In [ ]:
#///////////////////////Codigo Funcionando////////////////////////

import cv2

cap = cv2.VideoCapture("ojoFullHD1.mp4")
while True:
    red, frame = cap.read()
    
    roi = frame[100:600, 800:1400]
    gray_roi = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_roi = cv2.GaussianBlur(gray_roi, (7,7),0)
    _, threshold = cv2.threshold(gray_roi, 5, 255, cv2.THRESH_BINARY_INV)
    contornos,_=cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]

    
    for cnt in contornos:
        (x, y, w, h) = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        break
    
# Para dibujar los contornos encontrados
#     frame = cv2.drawContours(frame, contornos, -1, (0,255,0),2)
    
    cv2.imshow("threshold",frame) 

    key = cv2.waitKey(30)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import dlib
from math import hypot

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def puntoMedio(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)


while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
        landmarks = predictor(gray, face)
        left_point = (landmarks.part(36).x, landmarks.part(36).y)
        right_point = (landmarks.part(39).x, landmarks.part(39).y)
        center_top = puntoMedio(landmarks.part(37), landmarks.part(38)) 
        center_bottom = puntoMedio(landmarks.part(41), landmarks.part(40))
        
#         hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
#         ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)
        
        #sacamos el tamaño 
        hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
        ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
        
#         print(ver_line_lenght)


        # Para Recortar el ojo 
    
        
        
        
        relacion = hor_line_lenght / ver_line_lenght
#         print(relacion)
        
        if relacion > 5.5:
            cv2.putText(frame, "HOLA MUNDO", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,0,0) )
            
            
            
        left_eye_region = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                            (landmarks.part(37).x, landmarks.part(37).y),
                            (landmarks.part(38).x, landmarks.part(38).y),
                            (landmarks.part(39).x, landmarks.part(39).y),
                            (landmarks.part(40).x, landmarks.part(40).y),
                            (landmarks.part(41).x, landmarks.part(41).y)], np.int32)
                        
#         print(left_eye_region) 
#         cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
    
    #sacamos los puntos minimos y maximos para poder recortarlo
        min_x = np.min(left_eye_region[:, 0])
        max_x = np.max(left_eye_region[:, 0])
        min_y = np.min(left_eye_region[:, 1])
        max_y = np.max(left_eye_region[:, 1])
        
#         Recortamos 

        eye = frame[min_y: max_y, min_x: mafatkunx_x]
        
        gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
        
        _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
        
#       Redimensionamos el tamaño
        eye = cv2.resize(gray_eye, None, fx=5, fy=5)
        cv2.imshow("Eye", eye)
        
    
    
    cv2.imshow("frame",frame) 

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows() 

In [ ]:
import cv2
import numpy as np
import dlib
from math import hypot

In [ ]:
cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
def puntoMedio(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

In [ ]:
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
        landmarks = predictor(gray, face)
        
                
        if relacion > 5.5:
            cv2.putText(frame, "HOLA MUNDO", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0) )
            
    
   

In [ ]:
#Sacamos los Puntos para dibujar el contorno
            left_eye_region = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                                        (landmarks.part(37).x, landmarks.part(37).y),
                                        (landmarks.part(38).x, landmarks.part(38).y),
                                        (landmarks.part(39).x, landmarks.part(39).y),
                                        (landmarks.part(40).x, landmarks.part(40).y),
                                        (landmarks.part(41).x, landmarks.part(41).y)], np.int32)

In [ ]:
# Dibujamos el contorno
#         print(left_eye_region) 
        cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)

In [ ]:
            #sacamos los puntos minimos y maximos para poder recortarlo
            min_x = np.min(left_eye_region[:, 0])
            max_x = np.max(left_eye_region[:, 0])
            min_y = np.min(left_eye_region[:, 1])
            max_y = np.max(left_eye_region[:, 1])

In [ ]:
gray_eye = left_eye[min_y: max_y, min_x: max_x]

In [ ]:
 cv2.imshow("frame",frame) 

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows() 